# Grab all the loopy/inlining/odd shapes
Select a couple of examples to see what the pattern is for `stop_sequence` and `shape_meters`.

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'

import dask.dataframe as dd
import folium
import geopandas as gpd
import pandas as pd
import numpy as np
import shapely
import sys

from segment_speed_utils.project_vars import SEGMENT_GCS, analysis_date

# Temp append so we can import functions from scripts/
sys.path.append("./scripts/")
import prep_stop_segments
import cut_stop_segments
import loop_utils
import loopy

shape_with_utc = "f765b9d12fcca0173b4e3ddbc0374d18"
another_shape = "007cf76062f5957d4e38ea54e624c0ad"

In [2]:
loop_shapes = loop_utils.grab_loop_shapes(analysis_date)

stop_times_with_geom = prep_stop_segments.stop_times_aggregated_to_shape_array_key(
        analysis_date, loop_shapes)

st_loops = stop_times_with_geom.compute()

In [3]:
gdf = (loop_utils.assign_visits_to_stop(st_loops)
       .sort_values(["shape_array_key", "stop_sequence"])
       .reset_index(drop=True)
      )

In [4]:
gdf2 = gdf[gdf.shape_array_key.isin([another_shape])]

In [5]:
gdf2.head()

,shape_array_key,geometry,feed_key,name,stop_id,stop_sequence,stop_name,stop_geometry,num_visits
0,007cf76062f5957d4e38ea54e624c0ad,"LINESTRING (267265.540 -584480.665, 267265.976...",a7ba6f075198e9bf9152fab6c7faf0f6,San Diego Schedule,99437,1,13 St & Broadway,POINT (267258.726 -584480.878),2
1,007cf76062f5957d4e38ea54e624c0ad,"LINESTRING (267265.540 -584480.665, 267265.976...",a7ba6f075198e9bf9152fab6c7faf0f6,San Diego Schedule,99342,2,Broadway & Park Bl,POINT (267153.629 -584488.294),1
2,007cf76062f5957d4e38ea54e624c0ad,"LINESTRING (267265.540 -584480.665, 267265.976...",a7ba6f075198e9bf9152fab6c7faf0f6,San Diego Schedule,13311,3,Broadway & 8th Av,POINT (266847.851 -584503.046),1
3,007cf76062f5957d4e38ea54e624c0ad,"LINESTRING (267265.540 -584480.665, 267265.976...",a7ba6f075198e9bf9152fab6c7faf0f6,San Diego Schedule,13312,4,Broadway & 5th Av,POINT (266591.896 -584512.306),1
4,007cf76062f5957d4e38ea54e624c0ad,"LINESTRING (267265.540 -584480.665, 267265.976...",a7ba6f075198e9bf9152fab6c7faf0f6,San Diego Schedule,13313,5,Broadway & 2nd Av,POINT (266334.471 -584519.095),1


In [6]:
shape_geometry = gdf2.geometry.iloc[0]
stop_geometry_array = np.array(gdf2.stop_geometry)
stop_sequence_array = np.array(gdf2.stop_sequence)

In [7]:
current_stop_seq = 21

test_segment, test_stops = loopy.super_project(
    current_stop_seq,
    shape_geometry,
    stop_geometry_array,
    stop_sequence_array,
)

segment_to_plot, stops_to_plot = loopy.stop_segment_components_to_geoseries(
    test_segment, test_stops)

loopy.plot_segments_and_stops(segment_to_plot, stops_to_plot)

/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point


In [8]:
current_stop_seq = 8

test_segment, test_stops = loopy.super_project(
    current_stop_seq,
    shape_geometry,
    stop_geometry_array,
    stop_sequence_array,
)

segment_to_plot, stops_to_plot = loopy.stop_segment_components_to_geoseries(
    test_segment, test_stops)

loopy.plot_segments_and_stops(segment_to_plot, stops_to_plot)

/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point


## Expand to multiple stops within same shape

In [14]:
np.seterr(all="ignore")

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [53]:
subset_df = gdf2[gdf2.shape_array_key==another_shape
                ].reset_index(drop=True)

shape_geometry = subset_df.geometry.iloc[0]
stop_geometry_array = np.array(subset_df.stop_geometry)
stop_sequence_array = np.array(subset_df.stop_sequence)

segment_results = []

for stop_seq in stop_sequence_array:
        
    segment_i, _ = loopy.super_project(
        stop_seq,
        shape_geometry,
        stop_geometry_array,
        stop_sequence_array,
    )
    segment_i_series = loopy.stop_segment_components_to_geoseries(
        segment_i, [])
    
    segment_results.append(segment_i_series)


/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9

In [54]:
stop_segments = pd.concat(segment_results, 
                          axis=0).reset_index(drop=True)

In [55]:
subset_df = subset_df.assign(
    stop_segments_geometry = stop_segments
)

In [56]:
subset_df[subset_df.stop_segments_geometry.isna()]

,shape_array_key,geometry,feed_key,name,stop_id,stop_sequence,stop_name,stop_geometry,num_visits,stop_segments_geometry


## Expand to multiple shapes

In [46]:
gdf3 = gdf[gdf.shape_array_key.isin([shape_with_utc, another_shape])]
results_df = gpd.GeoDataFrame()

for one_shape in gdf3.shape_array_key.unique():
    subset_df = gdf3[gdf3.shape_array_key == one_shape
                    ].reset_index(drop=True)
    
    shape_geometry = subset_df.geometry.iloc[0]
    stop_geometry_array = np.array(subset_df.stop_geometry)
    stop_sequence_array = np.array(subset_df.stop_sequence)
    
    segment_results = []
    
    for stop_seq in stop_sequence_array:
        segment_i, _ = loopy.super_project(
        stop_seq,
        shape_geometry,
        stop_geometry_array,
        stop_sequence_array,
    )
    segment_i_series = loopy.stop_segment_components_to_geoseries(
        segment_i, [])
    
    segment_results.append(segment_i_series)
        
    stop_segments = pd.concat(segment_results, axis=0).reset_index(drop=True)

    subset_df = subset_df.assign(
        stop_segments = stop_segments
    )
    


/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9/site-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
/opt/conda/lib/python3.9

In [50]:
results_df[results_df.stop_segments.isna()].shape

(45, 10)

In [51]:
results_df.shape

(47, 10)